In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj

In [2]:
# datajoint configuration
dj.config['database.host'] = '127.0.0.1'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
# we will go into this attribute later
dj.config['fetch_format'] = 'frame'

In [3]:
connection = dj.conn()
connection

Connecting root@127.0.0.1:3306


DataJoint connection (connected) root@127.0.0.1:3306

In [4]:
dj.config.save_local()

In [5]:
dj.config.save_global()

In [6]:
dj.set_password('simple')

Password updated.
Update local setting? [yes, no]: yes
Saved settings in dj_local_conf.json


In [7]:
recording_schema = dj.schema('recording_schema_alvaro')

In [ ]:
@recording_schema
class RecordingInfoAlvaro(dj.Manual):
    definition="""
    # recording info alvaro
    recording_id : int # fly number that day, not subject_id
    ---
    experimenter = 'ASD' : varchar(3) # set to ASD by default but can be other experimenter 
    recording_date : int # yymmdd
    subject_id: varchar(20) # fly
    cell_number : int # cell recorded and corresponding to recording_id
    subrecording_id : int # correspond to each different recording
    genotype_id : int
    stim_path : varchar(2000) # path to location of the stimulus file
    recording_path : varchar(2000) # path to location of recording trace
    cell_type: varchar(200) 
    age : float # age of fly in days
    fly_sex = 'U': enum('F', 'M', 'U')  # sex of fly
    quality : varchar (200)
    internal_solution : varchar (200)
    bath_solution : varchar (200)
    brain_location : varchar (200)
    comments = null : varchar(4000)
    """
    
    
@recording_schema
class StimulusType(dj.Lookup):
    definition="""
    # type of stimuli
    ->RecordingInfoAlvaro
    ---
    stimulus_type: varchar(50) :('FFF', 'SRF', 'Bars')
    wavelength:
    intensity_list:
    background:
    
    """

@recording_schema
class Traces(dj.Imported):
    definition="""
    # Recording traces 
    ->subrecording_id
    ---
    raw_trace: longblob
    photodiode_trace: longblob
    psths: longblob
    stimulus_timecourse: longblog 
    Stimulus: ‘FFF’ (extracted from metadata)
    """
        
@recording_schema
class Traces_preprocessing(dj.Manual):
    definition="""
    # Preprocessing criteria for traces
    ->Traces
    ---
    filter: 
    aligned_events:
    wavelength:
    intensity:
    """

@recording_schema
class Processed(dj.Computed):
    definition="""
    Store processed data
    """



@recording_schema
class Analyzed(dj.Computed):
    definition="""
    #Store analyzed data
    """










    
    
        
@recording_schema
class MembraneProperties(dj.Computed):
    definition = """
    ->RecordingInfo
    ---
    
    """
@recording_schema
class FFFAlignmentParams(dj.Computed):
    definition = """
    ->RecordingInfo
    ---
    alignment_method='number':varchar(255)
    sigma=null: double
    number=null: varchar(255)
    """

@recording_schema
class FFFData(dj.Computed,dj.BaseMixin):
    # This table stores Full Field Flash (FFF) response data

    definition = """
    ->RecordingInfo
    ---
    on_repeats=null: longblob # this is a numpy array of response to ON stim
    off_repeats=null: longblob # this is a numpy array of response to OFF stim
    """

@recording_schema
class SRFData (dj.Computed,??)

@recording_schema
class BRFData (dj.Computed,??)


